In [17]:
import os
import numpy as np
import mne
import pickle
import pandas as pd

In [18]:
dataset_dir = "./dataset/raw_dataset/"
output_dir = "./dataset/preprocessed_dataset/"
window_size = 10
start_subject = 1
end_subject = 4

In [20]:
def data_1Dto2D(data, Y=10, X=11):
	data_2D = np.zeros([Y, X])
	# data_2D[0] = ( 	   	 0, 	   0,  	   	 0, 	   0, data[21], data[22], data[23], 	   0,  	     0, 	   0, 	 	 0) 
	# data_2D[1] = (	  	 0, 	   0,  	   	 0, data[24], data[25], data[26], data[27], data[28], 	   	 0,   	   0, 	 	 0) 
	# data_2D[2] = (	  	 0, data[29], data[30], data[31], data[32], data[33], data[34], data[35], data[36], data[37], 	 	 0) 
	# data_2D[3] = (	  	 0, data[38],  data[0],  data[1],  data[2],  data[3],  data[4],  data[5],  data[6], data[39], 		 0) 
	# data_2D[4] = (data[42], data[40],  data[7],  data[8],  data[9], data[10], data[11], data[12], data[13], data[41], data[43]) 
	# data_2D[5] = (	  	 0, data[44], data[14], data[15], data[16], data[17], data[18], data[19], data[20], data[45], 		 0) 
	# data_2D[6] = (	  	 0, data[46], data[47], data[48], data[49], data[50], data[51], data[52], data[53], data[54], 		 0) 
	# data_2D[7] = (	  	 0, 	   0, 	 	 0, data[55], data[56], data[57], data[58], data[59], 	   	 0, 	   0, 		 0) 
	# data_2D[8] = (	  	 0, 	   0, 	 	 0, 	   0, data[60], data[61], data[62], 	   0, 	   	 0, 	   0, 		 0) 
	# data_2D[9] = (	  	 0, 	   0, 	 	 0, 	   0, 	     0, data[63], 		 0, 	   0, 	   	 0, 	   0, 		 0) 
	data_2D[0] = ( 	   	 0, 	   0,  	   	 0, 	   0,        0,        0,        0, 	   0,  	     0, 	   0, 	 	 0) 
	data_2D[1] = (	  	 0, 	   0,  	   	 0, data[24],        0,        0,        0, data[28], 	   	 0,   	   0, 	 	 0) 
	data_2D[2] = (	  	 0,        0,        0,        0,        0,        0,        0,        0,        0,        0, 	 	 0) 
	data_2D[3] = (	  	 0,        0,        0,        0,        0,        0,        0,        0,        0,        0, 		 0) 
	data_2D[4] = (       0,        0,        0,        0,        0,        0,        0,        0,        0,        0,        0) 
	data_2D[5] = (	  	 0, data[44],        0,        0,        0,        0,        0,        0,        0, data[45], 		 0) 
	data_2D[6] = (	  	 0,        0,        0,        0,        0,        0,        0,        0,        0,        0, 		 0) 
	data_2D[7] = (	  	 0, 	   0, 	 	 0,        0,        0,        0,        0,        0, 	   	 0, 	   0, 		 0) 
	data_2D[8] = (	  	 0, 	   0, 	 	 0, 	   0,        0,        0,        0, 	   0, 	   	 0, 	   0, 		 0) 
	data_2D[9] = (	  	 0, 	   0, 	 	 0, 	   0, 	     0,        0, 		 0, 	   0, 	   	 0, 	   0, 		 0) 
	return data_2D

In [21]:
def windows(data, size):
	start = 0
	while ((start+size) < data.shape[0]):
		yield int(start), int(start + size)
		start += (size/2)

def segment_signal_without_transition(data, label, window_size):
	for (start, end) in windows(data, window_size):
		if((len(data[start:end]) == window_size) and (len(set(label[start:end]))==1)):
			if(start == 0):
				segments = data[start:end]
				# labels = stats.mode(label[start:end])[0][0]
				labels = np.array(list(set(label[start:end])))
			else:
				segments = np.vstack([segments, data[start:end]])
				labels = np.append(labels, np.array(list(set(label[start:end]))))
				# labels = np.append(labels, stats.mode(label[start:end])[0][0])
	return segments, labels

In [19]:
for i in range(start_subject, end_subject):
    if i == 89:
        continue
    data_dir = dataset_dir + "S" + format(i, "03d")
    task_list = [task for task in os.listdir(data_dir) if os.path.isdir(os.path.join(data_dir, task))]
    print(task_list)
    print(i)
    for task in task_list:
        if ("RO2" in task) or ("R04" in task) or ("R06" in task):
            print(task + " begin:")
            data_file 	= data_dir+"/"+task+"/"+task+".csv"
            label_file 	= data_dir+"/"+task+"/"+task+".label.csv"
            data		= pd.read_csv(data_file)
            label		= pd.read_csv(label_file)
            data_label	= pd.concat([data, label], axis=1)
            data_label	= data_label.loc[data_label['labels']!= 'rest']
            label		= data_label['labels']
            data_label.drop('labels', axis=1, inplace=True)
            data		= data_label.as_matrix()

['S001R06', 'S001R02', 'S001R12', 'S001R11', 'S001R05', 'S001R10', 'S001R13', 'S001R07', 'S001R04', 'S001R08', 'S001R14', 'S001R01', 'S001R09', 'S001R03']
1
S001R06 begin:
/home/amar/.local/lib/python3.6/site-packages/ipykernel_launcher.py:19: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
S001R04 begin:
['S002R04', 'S002R08', 'S002R07', 'S002R06', 'S002R14', 'S002R03', 'S002R13', 'S002R02', 'S002R09', 'S002R01', 'S002R12', 'S002R05', 'S002R10', 'S002R11']
2
S002R04 begin:
S002R06 begin:
['S003R05', 'S003R04', 'S003R07', 'S003R12', 'S003R01', 'S003R13', 'S003R09', 'S003R03', 'S003R02', 'S003R08', 'S003R11', 'S003R10', 'S003R14', 'S003R06']
3
S003R04 begin:
S003R06 begin:
